## Timeseries forecasting using RNN
* When dealing with time series (and other types of sequences such as sentences), the input features are generally represented as 3D arrays of shape [batch size, time steps, dimensionality]
  * dimensionality is 1 for univariate time series and more for multivariate time series

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# review np.random.rand
np.random.rand(4, 2, 1)

array([[[0.54679488],
        [0.34019394]],

       [[0.43361353],
        [0.49642858]],

       [[0.43365276],
        [0.52327518]],

       [[0.76029944],
        [0.35396854]]])

In [ ]:
# Generate timeseries data

def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
  return series[..., np.newaxis].astype(np.float32)

In [ ]:
# Create a training set, a validation set, and a test set using generate_time_series function:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
# X_train contains 7,000 time series (i.e., its shape is [7000, 50, 1]) 
# the targets are column vectors (e.g., y_train has a shape of [7000, 1]).
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]

#  X_valid contains 2,000 (from the 7,000th time series to the 8,999th)
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]

#  X_test contains 1,000 (from the 9,000th to the 9,999th)
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
# identify the baseline metrics using naive forecasting
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020570153

## # Implement simple RNN
* By default, recurrent layers in Keras only return the final output. 
* To make them return one output per time step, you must set
return_sequences=True

In [ ]:
# Implement simple RNN
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, 
                           input_shape=[None, 1])
])

In [ ]:
# compile the model
model.compile(loss='mean_squared_error', optimizer='rmsprop')

#check model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train the model
model.fit(X_train,y_train, 
          epochs=30, 
          validation_data=(X_valid, y_valid), 
          batch_size=16, 
          verbose=2)

Epoch 1/30
438/438 - 3s - loss: 0.0281 - val_loss: 0.0145 - 3s/epoch - 6ms/step
Epoch 2/30
438/438 - 2s - loss: 0.0118 - val_loss: 0.0117 - 2s/epoch - 4ms/step
Epoch 3/30
438/438 - 2s - loss: 0.0112 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 4/30
438/438 - 3s - loss: 0.0111 - val_loss: 0.0115 - 3s/epoch - 7ms/step
Epoch 5/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 5ms/step
Epoch 6/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 7/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 8/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 9/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 10/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 11/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 12/30
438/438 - 2s - loss: 0.0111 - val_loss: 0.0115 - 2s/epoch - 4ms/step
Epoch 13/30
438/438 - 2s - loss: 0.01

In [ ]:
# predict the model on test data
#result = tf.argmax(model.predict(tf.expand_dims(X_test, 0)), axis=1)
#print(result.numpy(), y_test)

In [ ]:
# implement deep RNN
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.SimpleRNN(1)
])

In [ ]:
# compile the model
model.compile(loss='mean_squared_error', optimizer='rmsprop')

#check model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, None, 20)          440       
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 20)          820       
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 1)                 22        
                                                                 
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# train the model
model.fit(X_train,y_train, 
          epochs=50, 
          validation_data=(X_valid, y_valid), 
          batch_size=16, 
          verbose=2)

Epoch 1/50
438/438 - 7s - loss: 0.0103 - val_loss: 0.0045 - 7s/epoch - 17ms/step
Epoch 2/50
438/438 - 5s - loss: 0.0036 - val_loss: 0.0039 - 5s/epoch - 12ms/step
Epoch 3/50
438/438 - 5s - loss: 0.0033 - val_loss: 0.0038 - 5s/epoch - 12ms/step
Epoch 4/50
438/438 - 5s - loss: 0.0032 - val_loss: 0.0034 - 5s/epoch - 12ms/step
Epoch 5/50
438/438 - 5s - loss: 0.0031 - val_loss: 0.0031 - 5s/epoch - 12ms/step
Epoch 6/50
438/438 - 5s - loss: 0.0031 - val_loss: 0.0033 - 5s/epoch - 12ms/step
Epoch 7/50
438/438 - 5s - loss: 0.0030 - val_loss: 0.0032 - 5s/epoch - 12ms/step
Epoch 8/50
438/438 - 5s - loss: 0.0030 - val_loss: 0.0032 - 5s/epoch - 12ms/step
Epoch 9/50
438/438 - 5s - loss: 0.0030 - val_loss: 0.0032 - 5s/epoch - 12ms/step
Epoch 10/50
438/438 - 5s - loss: 0.0029 - val_loss: 0.0029 - 5s/epoch - 13ms/step
Epoch 11/50
438/438 - 5s - loss: 0.0029 - val_loss: 0.0030 - 5s/epoch - 12ms/step
Epoch 12/50
438/438 - 5s - loss: 0.0028 - val_loss: 0.0031 - 5s/epoch - 12ms/step
Epoch 13/50
438/438 - 5s 